In [1]:
import pandas as pd, json, pickle
from multiprocessing import Pool

In [2]:
original_df = pd.read_csv('blood-samples/full-data.csv', index_col=0)

In [3]:
original_df.head()

,car_chr,car_start,car_end,car_size,car_intlogqvalue,car_foldchange,car_pvalue,car_qvalue,car_summit_to_start,car_sample_id,...,car_se_anno,car_methylation_anno,car_interaction_anno,car_commmon_snp_anno,Sample ID,Biosample type,Biosample name,Tissue type,Cancer type,Region number
0,chr1,9997,10802,805,1359,13.36316,138.88042,135.96225,204,25,...,0,0,1,5,Sample_0025,Primary Cell,CD56Bright natural killer cell,Blood,Normal,48025
1,chr1,20963,21340,377,108,5.73316,13.03911,10.84643,329,25,...,0,0,0,0,Sample_0025,Primary Cell,CD56Bright natural killer cell,Blood,Normal,48025
2,chr1,28785,29747,962,1636,18.39825,166.64635,163.63165,551,25,...,0,189,3,1,Sample_0025,Primary Cell,CD56Bright natural killer cell,Blood,Normal,48025
3,chr1,437449,437967,518,106,6.49730,12.88990,10.69930,515,25,...,0,0,0,0,Sample_0025,Primary Cell,CD56Bright natural killer cell,Blood,Normal,48025
4,chr1,540776,541301,525,263,9.65766,28.77254,26.39556,193,25,...,2,0,6,4,Sample_0025,Primary Cell,CD56Bright natural killer cell,Blood,Normal,48025


In [4]:
original_df.columns

Index(['car_chr', 'car_start', 'car_end', 'car_size', 'car_intlogqvalue',
       'car_foldchange', 'car_pvalue', 'car_qvalue', 'car_summit_to_start',
       'car_sample_id', 'car_region_id', 'car_gsm_id', 'car_risk_snp_anno',
       'car_eqtl_anno', 'car_crisps_anno', 'car_te_anno', 'car_se_anno',
       'car_methylation_anno', 'car_interaction_anno', 'car_commmon_snp_anno',
       'Sample ID', 'Biosample type', 'Biosample name', 'Tissue type',
       'Cancer type', 'Region number'],
      dtype='object')

In [5]:
genome = pickle.load(open('hg19.pickle', 'rb'))

In [6]:
genome.keys()

dict_keys(['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX', 'chrY'])

In [7]:
for key, s in genome.items():
    print(set(s))

{'G', 'C', 'T', 'N', 'A'}
{'G', 'C', 'T', 'N', 'A'}
{'G', 'C', 'T', 'N', 'A'}
{'G', 'C', 'T', 'N', 'A'}
{'G', 'C', 'T', 'N', 'A'}
{'G', 'C', 'T', 'N', 'A'}
{'G', 'C', 'T', 'N', 'A'}
{'G', 'C', 'T', 'N', 'A'}
{'G', 'C', 'T', 'N', 'A'}
{'G', 'C', 'T', 'N', 'A'}
{'G', 'C', 'T', 'N', 'A'}
{'G', 'C', 'T', 'N', 'A'}
{'G', 'C', 'T', 'N', 'A'}
{'G', 'C', 'T', 'N', 'A'}
{'G', 'C', 'T', 'N', 'A'}
{'G', 'C', 'T', 'N', 'A'}
{'G', 'C', 'T', 'N', 'A'}
{'G', 'C', 'T', 'N', 'A'}
{'G', 'C', 'T', 'N', 'A'}
{'G', 'C', 'T', 'N', 'A'}
{'G', 'C', 'T', 'N', 'A'}
{'G', 'C', 'T', 'N', 'A'}
{'G', 'C', 'T', 'N', 'A'}
{'G', 'C', 'T', 'N', 'A'}


In [8]:
sample25 = original_df[original_df['Sample ID'] == 'Sample_0025']

In [9]:
def generate_megaseq(sample_id):
    bigseq = []
    
    sample = original_df[original_df['Sample ID'] == sample_id]

    for idx, row in sample.iterrows():
        chromosome = row['car_chr']
        start = row['car_start']
        end = row['car_end']
        sequence = genome[chromosome][start:end+1]
        bigseq.append(sequence)
        #if idx < 10:
            #print(start, end)
            #print(end-start+1)
            #print(sequence)

    megaseq = 'Z'.join(bigseq)
    
    return megaseq

In [10]:
megaseq25 = generate_megaseq('Sample_0025')

In [11]:
samples_frame = pd.read_csv('Samples-Of-Interest.csv')

In [13]:
def f(x):
    return x*x

with Pool(5) as p:
    print(p.map(f, [1, 2, 3]))
    
sample_names = [row['Sample ID'] for idx, row in samples_frame.iterrows()]
    
    
with Pool(16) as wickedpool:
    megaseqs = wickedpool.map(generate_megaseq, sample_names)
    
def make_df(sample_name, mega):
    df_here = pd.DataFrame(columns=['Sample ID', 'sequence'])
    df_here.loc[1] = [sample_name, mega]
    return df_here

inputs_here = zip(sample_names, megaseqs)
    
with Pool(16) as anotherpool:
    dfs = anotherpool.starmap(make_df, inputs_here)
    

df_of_megaseqs = pd.concat(dfs)

'''
#df_of_megaseqs = pd.Dataframe(columns=['Sample ID', 'sequence'])
ms_dfs = []

for idx, row in samples_frame.iterrows():
    sample_mega = generate_megaseq(row['Sample ID'])
    df_here = pd.DataFrame(columns=['Sample ID', 'sequence'])
    df_here.loc[1] = [row['Sample ID'], sample_mega]
    ms_dfs.append(df_here)
    
df_of_megaseqs = pd.concat(ms_dfs)
'''

[1, 4, 9]


"\n#df_of_megaseqs = pd.Dataframe(columns=['Sample ID', 'sequence'])\nms_dfs = []\n\nfor idx, row in samples_frame.iterrows():\n    sample_mega = generate_megaseq(row['Sample ID'])\n    df_here = pd.DataFrame(columns=['Sample ID', 'sequence'])\n    df_here.loc[1] = [row['Sample ID'], sample_mega]\n    ms_dfs.append(df_here)\n    \ndf_of_megaseqs = pd.concat(ms_dfs)\n"

In [14]:
df_of_megaseqs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 452 entries, 1 to 1
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Sample ID  452 non-null    object
 1   sequence   452 non-null    object
dtypes: object(2)
memory usage: 10.6+ KB


In [15]:
mega25 = df_of_megaseqs[['sequence']].iloc[0]['sequence']

In [16]:
len(mega25)

33278645

In [17]:
set(mega25)

{'A', 'C', 'G', 'N', 'T', 'Z'}

In [18]:
for item in set(mega25):
    print(f'{item} count {mega25.count(item)}\t percent {"{:10.4f}".format(100*mega25.count(item)/len(mega25))} %')

G count 9303278	 percent    27.9557 %
C count 9282473	 percent    27.8932 %
T count 7318978	 percent    21.9930 %
Z count 48024	 percent     0.1443 %
N count 20	 percent     0.0001 %
A count 7325872	 percent    22.0137 %


In [19]:
df_of_megaseqs = df_of_megaseqs.set_index('Sample ID')
samples_frame = samples_frame.set_index('Sample ID')

In [20]:
df_of_megaseqs

,sequence
Sample ID,
Sample_0025,NNNTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCT...
Sample_0026,NNNTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCT...
Sample_0027,AACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACC...
Sample_0028,CCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCT...
Sample_0029,NNNTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCT...
...,...
Sample_1467,CCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCT...
Sample_1468,CCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCT...
Sample_1469,ACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCC...


In [21]:
df_of_megaseqs['Cancer type'] = samples_frame['Cancer type']

In [22]:
for t in df_of_megaseqs['Cancer type']:
    print(t)

Normal
Normal
Normal
Normal
Normal
Normal
Normal
Leukemia
Leukemia
Leukemia
Leukemia
Leukemia
Leukemia
Leukemia
Coronary artery disease
Coronary artery disease
Coronary artery disease
Coronary artery disease
Coronary artery disease
Coronary artery disease
Coronary artery disease
Coronary artery disease
Coronary artery disease
Coronary artery disease
Coronary artery disease
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
N

In [23]:
df_of_megaseqs

,sequence,Cancer type
Sample ID,,
Sample_0025,NNNTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCT...,Normal
Sample_0026,NNNTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCT...,Normal
Sample_0027,AACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACC...,Normal
Sample_0028,CCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCT...,Normal
Sample_0029,NNNTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCT...,Normal
...,...,...
Sample_1467,CCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCT...,Other
Sample_1468,CCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCT...,Other
Sample_1469,ACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCC...,Other


In [24]:
samples_frame

,Biosample type,Biosample name,Tissue type,Cancer type,Region number
Sample ID,,,,,
Sample_0025,Primary Cell,CD56Bright natural killer cell,Blood,Normal,48025
Sample_0026,Primary Cell,CD56Bright natural killer cell,Blood,Normal,50447
Sample_0027,Primary Cell,CD56Bright natural killer cell,Blood,Normal,47349
Sample_0028,Primary Cell,CD56Bright natural killer cell,Blood,Normal,46698
Sample_0029,Primary Cell,CD56Dim natural killer cell,Blood,Normal,40745
...,...,...,...,...,...
Sample_1467,Other,Macrophage,Blood,Other,10568
Sample_1468,Other,Monocyte,Blood,Other,6102
Sample_1469,Other,Monocyte,Blood,Other,8042


In [25]:
df_of_megaseqs.to_csv('megasequence-to-cancer.csv')

In [26]:
compression_opts = dict(method='zip', archive_name='megasequence-to-cancer.csv')
df_of_megaseqs.to_csv('megasequence-to-cancer.zip', compression=compression_opts)